<a href="https://colab.research.google.com/github/byahram/119/blob/master/preprocessing/prep_label%2Btext%2Bmeta_%EC%95%84%EB%9E%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, warnings, glob, tqdm, json, csv
import numpy as np
import pandas as pd

from tqdm import tqdm

# pandas 출력 옵션 및 경고 설정
pd.set_option("display.max_columns", 120)
warnings.filterwarnings("ignore")

# 구글 드라이브
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# # 기존 폴더 및 파일 완전 삭제 (하위 포함)
# !rm -rf "/content/raw"
# !rm -rf "/content/processed"
!rm -rf "/content/raw/SL_서울"

# # 0) 폴더 생성
# !mkdir -p "/content/raw/TL_인천"
# !mkdir -p "/content/raw/VL_인천"
!mkdir -p "/content/raw/SL_서울"
# !mkdir -p "/content/processed"

# # 1) 드라이브 ZIP을 로컬로 복사
# !cp "/content/drive/MyDrive/Colab Data/TL_인천.zip" "/content/raw/TL_인천.zip"
# !cp "/content/drive/MyDrive/Colab Data/VL_인천.zip" "/content/raw/VL_인천.zip"
!cp "/content/drive/MyDrive/Colab Data/SL_서울.zip" "/content/raw/SL_서울.zip"

# # 2) 로컬에서 압축 해제 (빠름)
# !unzip -q "/content/raw/TL_인천.zip" -d "/content/raw/TL_인천"
# !unzip -q "/content/raw/VL_인천.zip" -d "/content/raw/VL_인천"
!unzip -q "/content/raw/SL_서울.zip" -d "/content/raw/SL_서울"

# # 작업 후 결과를 한 번에 드라이브로 동기화
# # !rsync -a --info=progress2 "/content/TL_인천_processed/" "/content/drive/MyDrive/Colab Data/TL_인천_processed/"

In [ ]:
# # 기본 경로(BASE_DIR) 설정
RAW_DIR = "/content/raw"
PROCEED_DIR = "/content/processed"

# TL_DIR = RAW_DIR + "/TL_인천"
# VL_DIR = RAW_DIR + "/VL_인천"
SL_DIR = RAW_DIR + "/SL_서울"

# SAVE_TL = PROCEED_DIR + "/df_tl.csv"
# SAVE_TL_TS = PROCEED_DIR + "/df_tl_ts_merged.csv"
# SAVE_TL_TS_META = PROCEED_DIR + "/df_tl_ts_meta_merged.csv"

# SAVE_VL = PROCEED_DIR + "/df_vl.csv"
# SAVE_VL_VS = PROCEED_DIR + "/df_vl_vs_merged.csv"
# SAVE_VL_VS_META = PROCEED_DIR + "/df_vl_vs_meta_merged.csv"

SAVE_SL = PROCEED_DIR + "/df_sl.csv"
SAVE_SL_SS = PROCEED_DIR + "/df_sl_ss_merged.csv"
SAVE_SL_SS_META = PROCEED_DIR + "/df_sl_ss_meta_merged.csv"

## 1. TL/VL

In [ ]:
# 폴더 리스트
src_dirs = [
    # os.path.join(TL_DIR, "TL_인천_구급"),
    # os.path.join(TL_DIR, "TL_인천_구조"),
    # os.path.join(TL_DIR, "TL_인천_기타"),
    # os.path.join(TL_DIR, "TL_인천_화재"),
    # os.path.join(VL_DIR, "VL_인천_구급"),
    # os.path.join(VL_DIR, "VL_인천_구조"),
    # os.path.join(VL_DIR, "VL_인천_기타"),
    # os.path.join(VL_DIR, "VL_인천_화재"),
    os.path.join(SL_DIR, "구급"),
    os.path.join(SL_DIR, "구조"),
    os.path.join(SL_DIR, "기타"),
    os.path.join(SL_DIR, "화재"),
]

all_data = []

# 폴더 리스트 순회
for src in src_dirs:
    folder_name = os.path.basename(src)
    files = sorted(glob.glob(os.path.join(src, "*.json")))

    print(f"'{folder_name}' 내 JSON 파일 수: {len(files)}개")

    for file in tqdm(files, desc=f"Loading {folder_name}"):
        try:
            with open(file, "r", encoding="utf-8") as f:
                data = json.load(f)

            # JSON 구조 평탄화
            if isinstance(data, dict):
                df_all = pd.json_normalize(data)
            elif isinstance(data, list):
                df_all = pd.json_normalize(data)
            else:
                continue

            df_all["source_folder"] = folder_name  # 폴더명 기록
            df_all["filename"] = os.path.basename(file)  # 파일명 기록
            all_data.append(df_all)

        except Exception as e:
            print(f"'{file}' 읽기 실패: {e}")

# 리스트를 하나의 DataFrame으로 합치기
df_all = pd.concat(all_data, ignore_index=True)
df_all.shape

'구급' 내 JSON 파일 수: 75개


Loading 구급: 100%|██████████| 75/75 [00:00<00:00, 558.00it/s]


'구조' 내 JSON 파일 수: 15개


Loading 구조: 100%|██████████| 15/15 [00:00<00:00, 552.32it/s]


'기타' 내 JSON 파일 수: 3개


Loading 기타: 100%|██████████| 3/3 [00:00<00:00, 367.74it/s]


'화재' 내 JSON 파일 수: 7개


Loading 화재: 100%|██████████| 7/7 [00:00<00:00, 444.11it/s]


(100, 18)

In [ ]:
df = df_all.copy()
print(df.shape)
df.head()

(100, 18)


,_id,audioPath,recordId,status,startAt,endAt,utterances,mediaType,gender,address,disasterLarge,disasterMedium,urgencyLevel,sentiment,symptom,triage,source_folder,filename
0,651e464d69a4f266f062680c,20230925/Seoul/2022/20220101/converted_[202201...,1f691b041e81d50f916e,12,0,68160,"[{'id': '775283a1', 'startAt': 438, 'endAt': 1...",Mobile,M,서울특별시 노원구 상계동,구급,질병(중증 외),중,당황/난처,[],,구급,651e464d69a4f266f062680c_20220101.json
1,651e464d69a4f266f0626817,20230925/Seoul/2022/20220101/converted_[202201...,81067b14750ca669312f,12,0,131400,"[{'id': 'b39486a5', 'startAt': 513, 'endAt': 1...",Mobile,M,서울특별시 은평구 불광동,구급,질병(중증 외),하,당황/난처,[],,구급,651e464d69a4f266f0626817_20220101.json
2,651e464d69a4f266f0626829,20230925/Seoul/2022/20220101/converted_[202201...,fd8b1a6c6aad93c228de,12,0,71400,"[{'id': 'fc2db008', 'startAt': 9739, 'endAt': ...",Mobile,M,서울특별시 은평구 응암동,구급,질병(중증),상,당황/난처,[],,구급,651e464d69a4f266f0626829_20220101.json
3,651e464d69a4f266f0626833,20230925/Seoul/2022/20220101/converted_[202201...,eb6884595174379e507c,12,0,56520,"[{'id': '581cc4b3', 'startAt': 7100, 'endAt': ...",Mobile,M,서울특별시 영등포구 여의도동,구급,질병(중증 외),중,불안/걱정,[],,구급,651e464d69a4f266f0626833_20220101.json
4,651e464d69a4f266f0626835,20230925/Seoul/2022/20220101/converted_[202201...,37068d7663ebf261a018,12,0,66960,"[{'id': '3249acb9', 'startAt': 561, 'endAt': 1...",Mobile,M,서울특별시 성북구 돈암동,구급,질병(중증),상,불안/걱정,[기타통증],잠재응급증상,구급,651e464d69a4f266f0626835_20220101.json


In [ ]:
df["utterances"][1]

[{'id': 'b39486a5',
  'startAt': 513,
  'endAt': 1937,
  'text': '예 119입니다.',
  'speaker': 0},
 {'id': '6543b0e3',
  'startAt': 2254,
  'endAt': 8312,
  'text': '예, 저기 어, 이거 이 길거리에 웬 할아버지가 누워 계신데 건물 안쪽으로',
  'speaker': 1},
 {'id': 'wavesurfer_lsagr4ot0eo',
  'startAt': 7236,
  'endAt': 8409,
  'text': '어디에요?',
  'speaker': 0},
 {'id': 'wavesurfer_d5rf77reiso',
  'startAt': 8936,
  'endAt': 9664,
  'text': '예예',
  'speaker': 1},
 {'id': '0707626f',
  'startAt': 10159,
  'endAt': 11059,
  'text': '주소 알려주세요.',
  'speaker': 0},
 {'id': 'df64e8fd',
  'startAt': 12404,
  'endAt': 16934,
  'text': '주소가 여기 그 은평구 불광동 연신교회.',
  'speaker': 1},
 {'id': 'b9be99aa',
  'startAt': 18291,
  'endAt': 19181,
  'text': '연신교회요?',
  'speaker': 0},
 {'id': 'df7fafda',
  'startAt': 19571,
  'endAt': 23721,
  'text': '예, 연신교회 옆에 그 세븐일레븐 편의점 있거든요? 그 건물.',
  'speaker': 1},
 {'id': 'b51a1ed6',
  'startAt': 22807,
  'endAt': 24461,
  'text': '세븐 그 무슨 점이에요?',
  'speaker': 0},
 {'id': '9684b00e',
  'startAt': 25001,

In [ ]:
def join_utterance_texts(utterances):
    """
    utterances: list[dict]
    - 각 dict의 "text"만 추출
    - "startAt" 기준 정렬
    - "문장1", "문장2", "문장3"
    """
    if not isinstance(utterances, list):
        return None

    # dict만 남기고, text 키 있는 것만
    items = [u for u in utterances if isinstance(u, dict) and ("text" in u)]
    if not items:
        return None

    # startAt로 정렬
    if all("startAt" in u for u in items):
        items = sorted(items, key=lambda x: x.get("startAt", 0))

    quoted = []

    for u in items:
        t = u.get("text")
        if t is None:
            continue
        t = str(t).replace("\n", " ").strip()
        if not t:
            continue
        t = t.replace('"', '\\"')
        quoted.append(f'"{t}"')

    return ", ".join(quoted) if quoted else None

In [ ]:
df_text = pd.DataFrame({
    "_id": df["_id"].values,
    "audioPath": df["audioPath"].values,
    "recordId": df["recordId"].values,
    "text": df["utterances"].apply(join_utterance_texts).values,
})

df_text.head()

,_id,audioPath,recordId,text
0,651e464d69a4f266f062680c,20230925/Seoul/2022/20220101/converted_[202201...,1f691b041e81d50f916e,"""네, 119입니다, 말씀하세요."", ""예, 여기 지금 취객 한 분이 누워계셔서 길..."
1,651e464d69a4f266f0626817,20230925/Seoul/2022/20220101/converted_[202201...,81067b14750ca669312f,"""예 119입니다."", ""예, 저기 어, 이거 이 길거리에 웬 할아버지가 누워 계신..."
2,651e464d69a4f266f0626829,20230925/Seoul/2022/20220101/converted_[202201...,fd8b1a6c6aad93c228de,"""네, 여보세요."", ""아, 네, 안녕하세요. 여기 119인데요. 이거 남자분 쓰러..."
3,651e464d69a4f266f0626833,20230925/Seoul/2022/20220101/converted_[202201...,eb6884595174379e507c,"""여보세요."", ""예, 119예요. 저희 순복음 교회 예 순복음 교회 앞에 있는 버..."
4,651e464d69a4f266f0626835,20230925/Seoul/2022/20220101/converted_[202201...,37068d7663ebf261a018,"""네, 119입니다."", ""네, 안녕하세요. 여기 그 성복 산신 병원 근처인데요.""..."


In [ ]:
df_text["text"][1]

'"예 119입니다.", "예, 저기 어, 이거 이 길거리에 웬 할아버지가 누워 계신데 건물 안쪽으로", "어디에요?", "예예", "주소 알려주세요.", "주소가 여기 그 은평구 불광동 연신교회.", "연신교회요?", "예, 연신교회 옆에 그 세븐일레븐 편의점 있거든요? 그 건물.", "세븐 그 무슨 점이에요?", "편의점이요.", "편의점 무슨 점이냐고요, 그러니까", "가만 있어보자 그 연신교회 옆에 연신교회 옆에 작은 데 하나 있어요.", "세븐일레븐 무슨 점인지", "바깥은 바깥은 데 말고.", "무슨 점이냐고 써 있어요?", "연신교회 밖", "안 적혀있어요.", "안 적혀있어요?", "예, 세븐일레븐만 적혀 있고 그 두 군데 있는데, 큰 길거리면 그 안쪽에 있는데", "그 연신교회하고", "그 연신교회 바로 옆에 있어요. 그 건물에 그 세븐일레븐 건물에", "연신교회 옆이에요?", "예, 예, 그 세븐일레 건 그", "그 거기서 들어와 그 큰 길에서 들어오셔가지고.", "연신교회 쪽으로 들어오시면은", "저 그 고 그 건물에 할아버지가 누워 누워 계신데 지금 건물 안에", "연신교회 옆에 있는 세븐일레븐에 누워있다고요?", "예, 예, 예, 예예 세븐일레븐 건물에 뭐 나 이 이거. 지금 날이 영하 11도인데 지금", "연신교회 옆에 있는 세븐일레븐에 할아버지가 누워있다는 거죠? 의식 있어요?", "예예예", "예? 예 의식은 있는", "환자 의식 있어요?", "예, 신음을 신음을 내시는 거 보니까 의식은 있는 거 같은데.", "네, 차 나가볼게요.", "예"'

In [ ]:
df_text.to_csv(
    # SAVE_TL,     # TL
    # SAVE_VL,   # VL
    SAVE_SL,
    index=False,
    encoding="utf-8-sig",
    quoting=csv.QUOTE_MINIMAL
)

print("1. TL/VL 저장 완료!")

1. TL/VL 저장 완료!


## 2. TL/VL + TS/TL

In [ ]:
# df_label = pd.read_csv("/content/processed/df_tl.csv")
# df_label = pd.read_csv("/content/processed/df_vl.csv")
df_label = pd.read_csv("/content/processed/df_sl.csv")

# df_audio = pd.read_csv("/content/processed/train_split_summary.csv")
# df_audio = pd.read_csv("/content/processed/split_summary_list_ValidationData_SJ.csv")
df_audio = pd.read_csv("/content/processed/split_audio_sample_2.csv")

In [ ]:
print(df_label.shape)
print(df_audio.shape)

print(df_label["_id"].dtypes)
print(df_audio["_id"].dtypes)

print(df_label["_id"][0])
print(df_audio["_id"][0])

(100, 4)
(100, 2)
object
object
651e464d69a4f266f062680c
651e464d69a4f266f0626835


In [ ]:
df_label.head()

,_id,audioPath,recordId,text
0,651e464d69a4f266f062680c,20230925/Seoul/2022/20220101/converted_[202201...,1f691b041e81d50f916e,"""네, 119입니다, 말씀하세요."", ""예, 여기 지금 취객 한 분이 누워계셔서 길..."
1,651e464d69a4f266f0626817,20230925/Seoul/2022/20220101/converted_[202201...,81067b14750ca669312f,"""예 119입니다."", ""예, 저기 어, 이거 이 길거리에 웬 할아버지가 누워 계신..."
2,651e464d69a4f266f0626829,20230925/Seoul/2022/20220101/converted_[202201...,fd8b1a6c6aad93c228de,"""네, 여보세요."", ""아, 네, 안녕하세요. 여기 119인데요. 이거 남자분 쓰러..."
3,651e464d69a4f266f0626833,20230925/Seoul/2022/20220101/converted_[202201...,eb6884595174379e507c,"""여보세요."", ""예, 119예요. 저희 순복음 교회 예 순복음 교회 앞에 있는 버..."
4,651e464d69a4f266f0626835,20230925/Seoul/2022/20220101/converted_[202201...,37068d7663ebf261a018,"""네, 119입니다."", ""네, 안녕하세요. 여기 그 성복 산신 병원 근처인데요.""..."


In [ ]:
df_audio.head()

,_id,audio_split_list
0,651e464d69a4f266f0626835,"['651e464d69a4f266f0626835_20220101_0.wav', '6..."
1,651e464d69a4f266f0626864,"['651e464d69a4f266f0626864_20220101_0.wav', '6..."
2,651e464d69a4f266f0626833,"['651e464d69a4f266f0626833_20220101_0.wav', '6..."
3,651e464d69a4f266f062683c,"['651e464d69a4f266f062683c_20220101_0.wav', '6..."
4,651e464d69a4f266f062687f,"['651e464d69a4f266f062687f_20220101_0.wav', '6..."


In [ ]:
df_merge = df_audio.merge(df_label[["_id","audioPath","recordId","text"]], on="_id", how="left")

cols = ["_id", "audioPath", "recordId", "audio_split_list", "text"]
df_merge = df_merge.reindex(columns=cols)

print(df_merge.shape)
df_merge.head()

(100, 5)


,_id,audioPath,recordId,audio_split_list,text
0,651e464d69a4f266f0626835,20230925/Seoul/2022/20220101/converted_[202201...,37068d7663ebf261a018,"['651e464d69a4f266f0626835_20220101_0.wav', '6...","""네, 119입니다."", ""네, 안녕하세요. 여기 그 성복 산신 병원 근처인데요.""..."
1,651e464d69a4f266f0626864,20230925/Seoul/2022/20220101/converted_[202201...,a0dcc36ef3067aff94d1,"['651e464d69a4f266f0626864_20220101_0.wav', '6...","""네, 119입니다. 말씀하세요."", ""네, 여기."", ""네."", ""지금 영나루 로..."
2,651e464d69a4f266f0626833,20230925/Seoul/2022/20220101/converted_[202201...,eb6884595174379e507c,"['651e464d69a4f266f0626833_20220101_0.wav', '6...","""여보세요."", ""예, 119예요. 저희 순복음 교회 예 순복음 교회 앞에 있는 버..."
3,651e464d69a4f266f062683c,20230925/Seoul/2022/20220101/converted_[202201...,d0732a67d23412f18906,"['651e464d69a4f266f062683c_20220101_0.wav', '6...","""119입니다."", ""네, 지금 할아버지가 쓰러지셔가지고요."", ""거기 주소 좀 불..."
4,651e464d69a4f266f062687f,20230925/Seoul/2022/20220101/converted_[202201...,47af204992e3d9921daa,"['651e464d69a4f266f062687f_20220101_0.wav', '6...","""예 119입니다."", ""여보세요"", ""아 예, 여기 노량진 장성 사우나인데요."",..."


In [ ]:
# df_merge.to_csv(SAVE_TL_TS, index=False, encoding="utf-8-sig")
df_merge.to_csv(SAVE_SL_SS, index=False, encoding="utf-8-sig")
print("저장완료")

저장완료


In [ ]:
df_merge.columns

Index(['_id', 'audioPath', 'recordId', 'audio_split_list', 'text'], dtype='object')

In [ ]:
df_merge.shape

(100, 5)

## 3. TL/VL + TS/VS + T_meta/V_meta

In [ ]:
# df_label_text = pd.read_csv("/content/processed/df_tl_ts_merged.csv")
df_label_text = pd.read_csv("/content/processed/df_sl_ss_merged.csv")

# df_meta = pd.read_csv("/content/processed/df_train_meta_output.csv")
df_meta = pd.read_csv("/content/processed/df_sample_meta_output.csv")

In [ ]:
print(df_label_text.shape)
print(df_meta.shape)

print(df_label_text.columns)
print(df_meta.columns)

print(df_label_text["_id"].dtypes)
print(df_meta["id"].dtypes)

print(df_label_text["_id"][0])
print(df_meta["id"][0])

(100, 5)
(100, 11)
Index(['_id', 'audioPath', 'recordId', 'audio_split_list', 'text'], dtype='object')
Index(['id', 'recordId', 'mediaType', 'gender', 'address', 'disasterLarge',
       'disasterMedium', 'urgencyLevel', 'sentiment', 'symptom', 'triage'],
      dtype='object')
object
object
651e464d69a4f266f0626835
651e464d69a4f266f0626a0e


In [ ]:
# 두 DF 병합 (_id와 id 기준)
df_merged = pd.merge(
    df_label_text,
    df_meta,
    left_on="_id",
    right_on="id",
    how="left"   # left join: label_text 기준으로 유지
)

# 중복 id 컬럼 제거
df_merged = df_merged.drop(columns=["id"])

df_merged.shape

(100, 15)

In [ ]:
df_merged.columns

Index(['_id', 'audioPath', 'recordId_x', 'audio_split_list', 'text',
       'recordId_y', 'mediaType', 'gender', 'address', 'disasterLarge',
       'disasterMedium', 'urgencyLevel', 'sentiment', 'symptom', 'triage'],
      dtype='object')

In [ ]:
# df_merged.to_csv("/content/processed/df_tl_ts_meta_merged.csv", index=False, encoding="utf-8-sig")
# df_merged.to_csv("/content/processed/df_vl_vs_meta_merged.csv", index=False, encoding="utf-8-sig")
df_merged.to_csv("/content/processed/df_sl_ss_meta_merged.csv", index=False, encoding="utf-8-sig")
print("저장완료")

저장완료
